In [2]:
import uuid
import boto3
import pprint
import botocore
import logging

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)


In [7]:
# global constants
ENDPOINT_URL: str = None


In [ ]:
# you want to make sure that install sequence is as follows
# %pip install boto3-1.28.54-py3-none-any.whl
# %pip install botocore-1.31.54-py3-none-any.whl
# %pip install awscli-1.29.54-py3-none-any.whl

# exit out if the Boto3 (Python) SDK versions are not correct
assert boto3.__version__ == "1.28.73"
assert botocore.__version__ == "1.31.73"


In [8]:
input_text:str = "What are the XGBoost versions supported in Amazon SageMaker?" # replace this with a prompt relevant to your agent
agent_id:str = 'WOSQXZH6WD' # note this from the agent console on Bedrock
agent_alias_id:str = 'MEF1YZ6SIB' # fixed for draft version of the agent
session_id:str = str(uuid.uuid1()) # random identifier
enable_trace:bool = True


In [12]:
# create an boto3 bedrock agent client
client = boto3.client("bedrock-agent-runtime")
logger.info(client)


[2024-07-06 20:03:13,221] p15839 {273891072.py:3} INFO - <botocore.client.AgentsforBedrockRuntime object at 0x7f5bf4b433d0>


In [13]:
# invoke the agent API
response = client.invoke_agent(inputText=input_text,
    agentId=agent_id,
    agentAliasId=agent_alias_id,
    sessionId=session_id,
    enableTrace=enable_trace
)

logger.info(pprint.pprint(response))


[2024-07-06 20:03:14,148] p15839 {4226590062.py:9} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Sat, 06 Jul 2024 20:03:14 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '8fe82702-3bd2-11ef-a3c5-0afff4e8c80d',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '03f257bb-beb4-40a9-a90d-d69443c24da0'},
                      'HTTPStatusCode': 200,
                      'RequestId': '03f257bb-beb4-40a9-a90d-d69443c24da0',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f5c04027850>,
 'contentType': 'application/json',
 'sessionId': '8fe82702-3bd2-11ef-a3c5-0afff4e8c80d'}


In [11]:
%%time
import json
event_stream = response['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}") 
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)


[2024-07-06 20:02:24,906] p15839 {<timed exec>:11} INFO - {
  "agentAliasId": "MEF1YZ6SIB",
  "agentId": "WOSQXZH6WD",
  "agentVersion": "1",
  "sessionId": "8fe82702-3bd2-11ef-a3c5-0afff4e8c80d",
  "trace": {
    "orchestrationTrace": {
      "rationale": {
        "text": "",
        "traceId": "012146a9-0e32-4f5f-aef2-3c2de5d43e18-0"
      }
    }
  }
}
[2024-07-06 20:02:24,907] p15839 {<timed exec>:11} INFO - {
  "agentAliasId": "MEF1YZ6SIB",
  "agentId": "WOSQXZH6WD",
  "agentVersion": "1",
  "sessionId": "8fe82702-3bd2-11ef-a3c5-0afff4e8c80d",
  "trace": {
    "orchestrationTrace": {
      "invocationInput": {
        "invocationType": "KNOWLEDGE_BASE",
        "knowledgeBaseLookupInput": {
          "knowledgeBaseId": "BAZR84ZGHA",
          "text": "What are the XGBoost versions supported in Amazon SageMaker?"
        },
        "traceId": "012146a9-0e32-4f5f-aef2-3c2de5d43e18-0"
      }
    }
  }
}
[2024-07-06 20:02:24,909] p15839 {<timed exec>:11} INFO - {
  "agentAliasId": "

CPU times: user 7.77 ms, sys: 4 ms, total: 11.8 ms
Wall time: 10.1 ms
